<a href="https://colab.research.google.com/github/Burhan1310/Deep-Learning/blob/main/Iris_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

**Create a Model Class that Inherits nn.Module**

In [4]:
class Model(nn.Module):
  def __init__(self, input_features=4, hidden1 = 8, hidden2 = 8, out_features = 3):
    super().__init__()
    self.fully_connected1 = nn.Linear(input_features, hidden1)
    self.fully_connected2 = nn.Linear(hidden1, hidden2)
    self.out = nn.Linear(hidden2, out_features)

  def forward(self, x):
    x = F.relu(self.fully_connected1(x))
    x = F.relu(self.fully_connected2(x))
    x = self.out(x)

    return x

**Pick a manual seed for randomization**

In [5]:
torch.manual_seed(41)
model = Model()